## Import Libraries and Data

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys 
sys.path.append('../../')

In [3]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
torch.cuda.set_device(0)

In [5]:
torch.cuda.is_available()

True

In [6]:
PATH = "/home/paperspace/data/petfinder/"
%time df_raw = pd.read_csv(f'{PATH}train.csv')

CPU times: user 96 ms, sys: 16 ms, total: 112 ms
Wall time: 109 ms


In [7]:
%time df_test = pd.read_csv(f'{PATH}test.csv')

CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 28.5 ms


In [8]:
os.listdir(PATH)

['train.zip',
 'sample_submission.csv',
 'breed_labels.csv',
 'train_images.zip',
 'submission.csv',
 '.ipynb_checkpoints',
 'color_labels.csv',
 'train_metadata',
 'train_sentiment.zip',
 'train_images',
 'test_images.zip',
 'test_sentiment',
 'train_sentiment',
 'test_sentiment.zip',
 'train_metadata.zip',
 'train.csv',
 'test.csv',
 'test_metadata.zip',
 'state_labels.csv',
 'test.zip']

In [9]:
df_raw.head().T

,0,1,2,3,4
Type,2,2,1,1,1
Name,Nibble,No Name Yet,Brisco,Miko,Hunter
Age,3,1,1,4,1
Breed1,299,265,307,307,307
Breed2,0,0,0,0,0
Gender,1,1,1,2,1
Color1,1,1,2,1,1
Color2,7,2,7,2,0
Color3,0,0,0,0,0
MaturitySize,1,2,2,2,2


In [10]:
len(df_raw)

14993

## Inspect JSON Files

In [11]:
f = open('/home/paperspace/data/petfinder/train_sentiment/bec6cc26b.json', 'r')
json.load(f)

{'sentences': [{'text': {'content': 'Active and playful puppies.',
    'beginOffset': -1},
   'sentiment': {'magnitude': 0.7, 'score': 0.7}},
  {'text': {'content': 'Looking for a goodhome.', 'beginOffset': -1},
   'sentiment': {'magnitude': 0.1, 'score': 0.1}},
  {'text': {'content': 'Size of maturity should be either small or medium size.',
    'beginOffset': -1},
   'sentiment': {'magnitude': 0, 'score': 0}},
  {'text': {'content': 'No caging or chaining for long hour except for precautionary purposes.',
    'beginOffset': -1},
   'sentiment': {'magnitude': 0.2, 'score': -0.2}},
  {'text': {'content': 'Commitment to spay is compulsory.', 'beginOffset': -1},
   'sentiment': {'magnitude': 0.1, 'score': -0.1}}],
 'tokens': [],
 'entities': [{'name': 'puppies',
   'type': 'OTHER',
   'metadata': {},
   'salience': 0.37191662,
   'mentions': [{'text': {'content': 'puppies', 'beginOffset': -1},
     'type': 'COMMON'}]},
  {'name': 'Size',
   'type': 'OTHER',
   'metadata': {},
   'salienc

In [12]:
f2 = open('/home/paperspace/data/petfinder/train_metadata/456c7a365-1.json', 'r')
json.load(f2)

{'labelAnnotations': [{'mid': '/m/04rky',
   'description': 'mammal',
   'score': 0.9250979,
   'topicality': 0.9250979},
  {'mid': '/m/035qhg',
   'description': 'fauna',
   'score': 0.9236901,
   'topicality': 0.9236901},
  {'mid': '/m/01l7qd',
   'description': 'whiskers',
   'score': 0.6884706,
   'topicality': 0.6884706},
  {'mid': '/m/0307l',
   'description': 'cat like mammal',
   'score': 0.645727,
   'topicality': 0.645727},
  {'mid': '/m/05nnm',
   'description': 'organism',
   'score': 0.6209524,
   'topicality': 0.6209524},
  {'mid': '/m/01yrx',
   'description': 'cat',
   'score': 0.61177486,
   'topicality': 0.61177486},
  {'mid': '/m/01j33z',
   'description': 'viverridae',
   'score': 0.60386884,
   'topicality': 0.60386884},
  {'mid': '/m/01280g',
   'description': 'wildlife',
   'score': 0.5414907,
   'topicality': 0.5414907}],
 'imagePropertiesAnnotation': {'dominantColors': {'colors': [{'color': {'red': 62,
      'green': 51,
      'blue': 28},
     'score': 0.26373

## Inspect Data

In [13]:
df_raw.shape

(14993, 24)

In [14]:
df_test.shape

(3948, 23)

In [15]:
df_raw.dtypes

Type               int64
Name              object
Age                int64
Breed1             int64
Breed2             int64
Gender             int64
Color1             int64
Color2             int64
Color3             int64
MaturitySize       int64
FurLength          int64
Vaccinated         int64
Dewormed           int64
Sterilized         int64
Health             int64
Quantity           int64
Fee                int64
State              int64
RescuerID         object
VideoAmt           int64
Description       object
PetID             object
PhotoAmt         float64
AdoptionSpeed      int64
dtype: object

In [16]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [17]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

AdoptionSpeed    0.000000
Age              0.000000
Breed1           0.000000
Breed2           0.000000
Color1           0.000000
Color2           0.000000
Color3           0.000000
Description      0.000800
Dewormed         0.000000
Fee              0.000000
FurLength        0.000000
Gender           0.000000
Health           0.000000
MaturitySize     0.000000
Name             0.083839
PetID            0.000000
PhotoAmt         0.000000
Quantity         0.000000
RescuerID        0.000000
State            0.000000
Sterilized       0.000000
Type             0.000000
Vaccinated       0.000000
VideoAmt         0.000000
dtype: float64

In [18]:
display_all(df_test.isnull().sum().sort_index()/len(df_raw))

Age             0.000000
Breed1          0.000000
Breed2          0.000000
Color1          0.000000
Color2          0.000000
Color3          0.000000
Description     0.000133
Dewormed        0.000000
Fee             0.000000
FurLength       0.000000
Gender          0.000000
Health          0.000000
MaturitySize    0.000000
Name            0.020209
PetID           0.000000
PhotoAmt        0.000000
Quantity        0.000000
RescuerID       0.000000
State           0.000000
Sterilized      0.000000
Type            0.000000
Vaccinated      0.000000
VideoAmt        0.000000
dtype: float64

In [19]:
df_raw.nunique()

Type                 2
Name              9060
Age                106
Breed1             176
Breed2             135
Gender               3
Color1               7
Color2               7
Color3               6
MaturitySize         4
FurLength            3
Vaccinated           3
Dewormed             3
Sterilized           3
Health               3
Quantity            19
Fee                 74
State               14
RescuerID         5595
VideoAmt             9
Description      14032
PetID            14993
PhotoAmt            31
AdoptionSpeed        5
dtype: int64

### Add JSON scores to DataFrame

In [20]:
%%time
senti_mag = {}
senti_score = {}
nf_count = 0
for pid in df_raw.PetID:    
    try:
        with open(f'{PATH}train_sentiment/{pid}.json', 'r') as f:
            sentiment = json.load(f)
        senti_mag[pid] = sentiment['documentSentiment']['magnitude']
        senti_score[pid] = sentiment['documentSentiment']['score']
    except FileNotFoundError:
        nf_count += 1
        senti_mag[pid] = np.NaN
        senti_score[pid] = np.NaN

CPU times: user 1.1 s, sys: 180 ms, total: 1.28 s
Wall time: 1.28 s


In [21]:
%%time
senti_mag_test = {}
senti_score_test = {}
nf_count_test = 0
for pid in df_test.PetID:    
    try:
        with open(f'{PATH}test_sentiment/{pid}.json', 'r') as f:
            sentiment = json.load(f)
        senti_mag_test[pid] = sentiment['documentSentiment']['magnitude']
        senti_score_test[pid] = sentiment['documentSentiment']['score']
    except FileNotFoundError:
        nf_count_test += 1
        senti_mag_test[pid] = np.NaN
        senti_score_test[pid] = np.NaN

CPU times: user 312 ms, sys: 24 ms, total: 336 ms
Wall time: 332 ms


In [22]:
df_raw.loc[:,'SentimentalAnalysisMagnitude']=senti_mag.values()
df_raw.loc[:,'SentimentalAnalysisScore']=senti_score.values()
df_raw.head(2).T

,0,1
Type,2,2
Name,Nibble,No Name Yet
Age,3,1
Breed1,299,265
Breed2,0,0
Gender,1,1
Color1,1,1
Color2,7,2
Color3,0,0
MaturitySize,1,2


In [23]:
df_test.loc[:,'SentimentalAnalysisMagnitude']=senti_mag_test.values()
df_test.loc[:,'SentimentalAnalysisScore']=senti_score_test.values()
df_test.head(2).T

,0,1
Type,1,2
Name,Puppy,London
Age,2,24
Breed1,307,266
Breed2,0,0
Gender,1,1
Color1,1,2
Color2,0,7
Color3,0,0
MaturitySize,2,2


In [24]:
df_raw.drop(['Description', 'Name', 'RescuerID', 'PetID'], axis=1, inplace = True)
df_test.drop(['Description', 'Name', 'RescuerID', 'PetID'], axis=1, inplace = True)

## Fit Random Forest Baseline

In [25]:
train_cats(df_raw)
train_cats(df_test)

In [26]:
df_raw.to_feather('pets_feather_raw')

In [27]:
%time df_raw = pd.read_feather('pets_feather_raw')

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.07 ms


In [28]:
df_raw.shape

(14993, 22)

In [29]:
df_test.shape

(3948, 21)

In [30]:
for n,c in df_raw.items(): 
    na_dict = fix_missing(df_raw, c, n, na_dict={})

In [31]:
for n,c in df_test.items(): 
    na_dict = fix_missing(df_test, c, n, na_dict={})

In [32]:
df_raw.SentimentalAnalysisMagnitude.fillna(value = df_raw.SentimentalAnalysisMagnitude.median(), inplace=True)
df_raw.SentimentalAnalysisScore.fillna(value = df_raw.SentimentalAnalysisScore.median(), inplace=True)

In [33]:
df_test.SentimentalAnalysisMagnitude.fillna(value = df_raw.SentimentalAnalysisMagnitude.median(), inplace=True)
df_test.SentimentalAnalysisScore.fillna(value = df_raw.SentimentalAnalysisScore.median(), inplace=True)

In [34]:
df, y = df_raw.drop('AdoptionSpeed', axis=1), df_raw.AdoptionSpeed

In [35]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 1000  # same as Kaggle's test set size
n_trn = len(df)-n_valid

X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((13993, 23), (13993,), (1000, 23))

In [36]:
X_train.sample(5).T

,13454,4748,3475,3147,9645
Type,2,1,1,1,2
Age,60,3,24,52,2
Breed1,292,307,307,119,266
Breed2,0,218,0,0,265
Gender,2,1,1,1,2
Color1,2,5,2,1,1
Color2,0,0,0,5,6
Color3,0,0,0,6,7
MaturitySize,3,1,2,2,1
FurLength,1,1,2,2,2


In [37]:
%%time
m = RandomForestClassifier(n_estimators=100, oob_score=True)
m.fit(X_train, y_train)

CPU times: user 2.06 s, sys: 80 ms, total: 2.14 s
Wall time: 2.13 s


In [38]:
m.oob_score_

0.4102051025512756

In [39]:
preds = m.predict(X_valid)

In [40]:
(preds == y_valid).mean()

0.418

## Make Predictions

In [41]:
preds = m.predict(df_test)

In [42]:
preds.shape

(3948,)

In [43]:
%time submission = pd.read_csv(f'{PATH}sample_submission.csv')

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.11 ms


In [44]:
submission.shape

(3948, 2)

In [45]:
submission.AdoptionSpeed = preds

In [46]:
!pwd

/home/paperspace/fastai/courses/dl1


In [47]:
submission.to_csv(path_or_buf = '/home/paperspace/data/petfinder/submission.csv')